In [1]:
import os
import sys
import cv2
from moviepy.editor import VideoFileClip, VideoClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

## avi => mp4 のプログラム

In [2]:
# 入力ファイルと出力ファイルを指定
input_file = "/taiga/junior/DADA_Datasets/DADA-2000/videos/images_10_001.avi"  # 入力Aviファイル名
output_file = "output.mp4"  # 出力Mp4ファイル名　このままでは作業ディレクトリに保存されます

# 動画情報を取得
video_clip = VideoFileClip(input_file)

# 解像度を取得
width = video_clip.size[0]
height = video_clip.size[1]
rotation = video_clip.rotation
fps = video_clip.fps
duration = video_clip.duration
print("video_clip.size:", width, height, rotation, fps, duration)

# avi動画をMP4に変換して保存
if rotation == 90:
    video_clip.write_videofile(output_file, codec='libx264', ffmpeg_params=["-vf", f"scale={height}:{width}"])
else:
    video_clip.write_videofile(output_file, codec='libx264', ffmpeg_params=["-vf", f"scale={width}:{height}"])

video_clip.size: 1584 660 0 30.0 29.27
Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4


## mp4の動画から異なる動画を分割

In [24]:
input_video_file = "output.mp4"

# 動画の切り替えを検出するための閾値を設定します
difference_threshold = 25000000  # 閾値は動画によって調整が必要かもしれません
min_time_between_switches = 2  # 最小の動画切り替え間隔（秒）

cap = cv2.VideoCapture(str(input_video_file))

# 動画の情報を取得します
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)  # 実際のフレームレートを取得します

# 前のフレームを初期化します
prev_frame = None

movie_switch_times = []

# フレームを読み込み、動画の切り替えを検出します
prev_switch_time = -1
for i in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break
    # グレースケールに変換します
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 前のフレームが存在する場合、差分を計算します
    if prev_frame is not None:
        frame_difference = cv2.absdiff(gray_frame, prev_frame)
        difference_sum = frame_difference.sum()
        #print(difference_sum)
        # 差分が閾値を超え，最小間隔以上の時間が経過した場合，動画切り替えとみなします
        if difference_sum > difference_threshold:
            current_time = i / frame_rate  # フレーム番号から実際の時間を計算
            if current_time - prev_switch_time >= min_time_between_switches:
                movie_switch_times.append(current_time)
                prev_switch_time = current_time
    prev_frame = gray_frame

cap.release()

print(f"切り替わりのタイミング : {movie_switch_times}s ")

切り替わりのタイミング : [14.666666666666666]s 


In [25]:
def split_video(file_path: str, split_points: list) -> None:
    # ファイル名から拡張子を取得
    file_name, file_extension = os.path.splitext(os.path.basename(file_path))

    # 分割したビデオの保存先ディレクトリを作成
    output_dir = f"{file_name}_split"
    os.makedirs(output_dir, exist_ok=True)

    # 動画の長さを取得
    with VideoFileClip(file_path) as video:
        total_duration = int(video.duration)

    # 修正したリストに終了時刻を追加して作業を行う
    modified_split_points = split_points + [total_duration]

    # リストの最初に0を追加
    modified_split_points.insert(0, 0)

    # タイミングを指定してビデオを分割
    for i, split_point in enumerate(modified_split_points[:-1]):
        start_time = split_point
        end_time = modified_split_points[i + 1]

        # 分割範囲を指定してビデオを切り出し
        output_path = os.path.join(output_dir, f"{file_name}_part_{i}{file_extension}")
        ffmpeg_extract_subclip(file_path, start_time, end_time, targetname=output_path)

In [26]:
# Usage example
input_video = "output.mp4"
split_video(input_video, movie_switch_times)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
